In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# from https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-05-CNN3/
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result 
  
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, w):
    #stides: 1, stride x, stride y, 1
    #padding: SAME, VALID
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    #ksize: 1, kernel width x, kernel width y, 1
    #strides=1, stride x, stride y, 1
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [7]:
xs = tf.placeholder(tf.float32, [None, 28*28])
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])
#print x_image.shape #[n_samples, 28, 28,1]

# 第一层卷积操作 ##  
# 第一二参数值得卷积核尺寸大小，即patch，第三个参数是图像通道数，第四个参数是卷积核的数目，代表会出现多少个卷积特征图像;  
#build the net
#kernel=5x5, input_size=1, output_size=32
w_conv1 = weight_variable([5,5,1,32])
# 对于每一个卷积核都有一个对应的偏置量。 
b_conv1 = bias_variable([32])
# 图片乘以卷积核，并加上偏执量，卷积结果28x28x32  
conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1) #output size 28*28*32
#     池化结果14x14x32 卷积结果乘以池化卷积核  
pool1 = max_pool_2x2(conv1)  #output size 14*14*32

# 第二层卷积操作 
# 32通道卷积，卷积出64个特征  
#kernel=5x5, input_size=32, output_size=64
w_conv2 = weight_variable([5,5,32,64])
# 64个偏执数据  
b_conv2 = bias_variable([64])
  # 注意h_pool1是上一层的池化结果，#卷积结果14x14x64  
conv2 = tf.nn.relu(conv2d(pool1, w_conv2) + b_conv2) #output size 14*14*64
pool2 = max_pool_2x2(conv2)  #output size 7*7*64
# 原图像尺寸28*28，第一轮图像缩小为14*14，共有32张，第二轮后图像缩小为7*7，共有64张 

## 第三层全连接操作 ##  
# 二维张量，第一个参数7*7*64的patch，也可以认为是只有一行7*7*64个数据的卷积，第二个参数代表卷积个数共1024个
#function1 layer
# 将第二层卷积池化结果reshape成只有一行7*7*64个数据 每一列都是一个图片特征  
pool3 = tf.reshape(pool2, [-1, 7*7*64]) #[n_samples, 7, 7, 64] >> [n_sample, 7*7*64]
w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
# 卷积操作，结果是1*1*1024，单行乘以单列等于1*1矩阵，
# matmul实现最基本的矩阵相乘，不同于tf.nn.conv2d的遍历相乘，自动认为是前行向量后列向量  
out3 = tf.nn.relu(tf.matmul(pool3, w_fc1) + b_fc1)

#function2 layer
    ## 第四层输出操作 ##  
    # 二维张量，1*1024矩阵卷积，共10个卷积，对应我们开始的ys长度为10  
    
w_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
# 最后的分类，结果为1*1*10 softmax和sigmoid都是基于logistic分类算法，一个是多分类一个是二分类  
prediction = tf.nn.softmax(tf.matmul(out3, w_fc2) + b_fc2)

#定义loss(最小误差概率)，选定优化优化loss
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))

loss_l2 = cross_entropy + 0.3*(tf.nn.l2_loss(w_fc2) + tf.nn.l2_loss(w_fc1))
 # 调用优化器优化，其实就是通过喂数据争取cross_entropy最小化    
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss_l2)

sess = tf.Session()

sess.run(tf.global_variables_initializer())

In [9]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
    if i%100 ==0:
        print ('step:',i,',','accuracy:',compute_accuracy(mnist.test.images, mnist.test.labels))

step: 0 , accuracy: 0.7135
step: 100 , accuracy: 0.8197
step: 200 , accuracy: 0.8778
step: 300 , accuracy: 0.8947
step: 400 , accuracy: 0.909
step: 500 , accuracy: 0.9133
step: 600 , accuracy: 0.9226
step: 700 , accuracy: 0.9248
step: 800 , accuracy: 0.9345
step: 900 , accuracy: 0.937
